In [5]:
import requests
import os

def detect_language_speechbrain_container(input_file, port=8001):
    url = f"http://localhost:{port}/identify_language"
    
    # Check if file exists
    if not os.path.isfile(input_file):
        print(f"Error: File {input_file} does not exist.")
        return None
    
    # Open file in binary mode
    with open(input_file, 'rb') as file:
        # Create a dictionary with the file, using 'audio' as the key
        files = {'audio': file}
        
        try:
            # Send POST request with file
            response = requests.post(url, files=files)
            response.raise_for_status()  # This will raise an exception for HTTP errors
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")
            return None

# Usage
BASE_DIR = "./standardized"  # Replace with your actual base directory
input_file = "ES2016a.Mix-Headset.wav"
port = 8001  # Replace with your actual port number

full_path = os.path.join(BASE_DIR, input_file)
result = detect_language_speechbrain_container(full_path, port)

if result:
    print(f"Detected language: {result}")
else:
    print("Failed to detect language")

An error occurred: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Failed to detect language
